In [59]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import os
import pandas as pd
from datetime import datetime
import time

from riotwatcher import LolWatcher, ApiError

""" 
API Rate Limits:
20 requests every 1 seconds(s)
100 requests every 2 minutes(s)
"""

watcher = LolWatcher(config['riotwatcher_api_key'])

In [60]:
# record start time
start_timer = time.time()

""" user-defined variables """
var_region = "euw1"
var_summoner_name= "AdAstraData"

""" fetch item_list from data_dragon and create dict with item_id : item_name """
data_dragon_versions = watcher.data_dragon.versions_for_region(region=var_region)
data_dragon_item_version = data_dragon_versions['n']['item']

item_list = watcher.data_dragon.items(data_dragon_item_version)

dict_item_id_item_name = {}

for item_id in item_list['data'].keys():
    df_item_id = pd.json_normalize(data=item_list['data'][item_id])
    item_name = df_item_id['name'].to_string(index=False)
    dict_item_id_item_name[item_id] = item_name

""" store summoner ids """
summoner_puuid = watcher.summoner.by_name(region=var_region, summoner_name=var_summoner_name)
my_puuid = summoner_puuid['puuid']

""" find all matches played by puuid """
last_matches_puuid = watcher.match.matchlist_by_puuid(
    region=var_region, 
    puuid=my_puuid, 
    count=100
    )

# record end time
end_timer = time.time()

# number of API calls
num_api_calls = 4

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 1.4801628589630127 s
Number of API calls: 4
API calls per second: 2.7


In [61]:
""" list of match_id for all matches played by summoner"""
len(last_matches_puuid)

100

In [62]:
dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

""" for each match_id """
for var_match_id in last_matches_puuid[0:100]:

    """ GET summoner puuids """
    watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)

    for puuid_num in range(len(watcher_match['metadata']['participants'])):
        puuid_features = watcher_match['info']['participants'][puuid_num]

        dict_match_info['match_id'].append(var_match_id)
        dict_match_info['team_id'].append(puuid_features['teamId'])
        dict_match_info['participant_puuid'].append(puuid_features['puuid'])
        dict_match_info['summoner_name'].append(puuid_features['summonerName'])
        dict_match_info['gold_earned'].append(puuid_features['goldEarned'])
    
    """ time delay with time.sleep """
    time.sleep(1)


In [63]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

list_all_match_participants = df_match_info['summoner_name'].unique()
len(list_all_match_participants)

853

In [64]:
len(list_all_match_participants)

853

In [70]:
list_all_match_participants[25:26]

array(['Alecostry22'], dtype=object)

In [77]:
""" user-defined variables """
var_region = "euw1"
# var_summoner_name= "AdAstraData"

dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

for var_participant_name in list_all_match_participants[27:30]:
    
    """ GET participant puuid """
    summoner_puuid = watcher.summoner.by_name(region=var_region, summoner_name=var_participant_name)
    var_participant_puuid = summoner_puuid['puuid']
    
    """ GET last 100 matches played by puuid """
    last_matches_puuid = watcher.match.matchlist_by_puuid(
        region=var_region, 
        puuid=var_participant_puuid, 
        count=100
    )

    """ time delay with time.sleep """
    time.sleep(1)

    """ for each match_id """
    for var_match_id in last_matches_puuid[0:100]:
        
        """ GET summoner puuids """
        watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)
        
        for puuid_num in range(len(watcher_match['metadata']['participants'])):
            puuid_features = watcher_match['info']['participants'][puuid_num]
            
            dict_match_info['match_id'].append(var_match_id)
            dict_match_info['team_id'].append(puuid_features['teamId'])
            dict_match_info['participant_puuid'].append(puuid_features['puuid'])
            dict_match_info['summoner_name'].append(puuid_features['summonerName'])
            dict_match_info['gold_earned'].append(puuid_features['goldEarned'])

        """ time delay with time.sleep """
        time.sleep(1)


# EUW1_5021326873


In [79]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,EUW1_6161945478,Blue,A2zEx9varFrcDbkKlqdGuD0dr5dCO0pcB4MRqH6N6Qha6J...,poutmasmurf,6898
1,EUW1_6161945478,Blue,MEVanfuJg9PZqtZbaJPB2JbkU9gbb83q9wDJobu06PXpnf...,FriiZoK7,9587
2,EUW1_6161945478,Blue,h3C486P2TXijAzf80wqo8wuscyhoo1Qypzd1pvu0JEYb6a...,haus300,8852
3,EUW1_6161945478,Blue,me53WpNeg3rqK2j0xb30wxMWFQ37MMZdF1RzPvSi38SR1j...,godbutbackwards,6209
4,EUW1_6161945478,Blue,Jl4DjLQgMwQK-9xga9-ddMeEgsq5NGUFy_Gy9O-SQkHQhW...,davidjimi,5610
...,...,...,...,...,...
1926,EUW1_5162267977,Blue,-C2u2z2GvgyjMrwuVLWcrkVN3joW392KZSMWtLRCHAO5v_...,popcgae,4749
1927,EUW1_5162267977,Blue,n9xTZEVS8IP4C0arauuyyHyrI_NvR0bz7D-uQV5rJdY9Nc...,Kanriee,7994
1928,EUW1_5162267977,Blue,joR0O9lnLuhRuU8z0OfKJxJU9xxOEguG-HCI_Yarrjm-Fp...,SyRuB1,14513
1929,EUW1_5162267977,Blue,GQIS690djRh5tNELDpJNdxcvyVlwGcqShzRI3krMOeJbjj...,skeith241,6349


In [80]:
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite:///riotwatcher_api_dt_match.db', echo=False)

# df_append = pd.DataFrame(
#     [['test', 'test', 'test', 'test', 'test']],
#     columns = ['match_id', 'team_id', 'participant_puuid', 'summoner_name', 'gold_earned']
# )

# df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)

for var_match_id in df_match_info['match_id'].unique():
    df_tmp = pd.read_sql_query("SELECT * from tbl_match_info WHERE match_id = '%s'" %(var_match_id), con=engine)
    
    if df_tmp.empty:
        df_append = df_match_info.query("match_id == @var_match_id")
        df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)
    else:
        pass


In [95]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info WHERE summoner_name = 'AdAstraData' AND gold_earned > 0", con=engine)
df['gold_earned'].astype(int).describe()
### AdAstraData

count      112.000000
mean      9048.589286
std       3601.078609
min        550.000000
25%       6694.000000
50%       8803.500000
75%      11406.500000
max      18333.000000
Name: gold_earned, dtype: float64

In [94]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info WHERE summoner_name = 'Storm and Sin' AND gold_earned > 0", con=engine)
df['gold_earned'].astype(int).describe()
### Storm and Sin

count      100.000000
mean     11145.490000
std       3752.877472
min        807.000000
25%       9047.500000
50%      11338.000000
75%      13439.500000
max      19373.000000
Name: gold_earned, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 4. Add new data and test if append respects existing match info

In [4]:
list_match_item_purchased = []

for var_match_id in last_matches_puuid[0:10]:

    """ store participant's puuids """
    dict_participants_puuids = {}

    match_participants = watcher.match.by_id(region=var_region, match_id=var_match_id)
    list_match_participants = match_participants['metadata']['participants']
    
    for idx in range(len(list_match_participants)):
        participant_id = idx + 1
        dict_participants_puuids[str(participant_id)] = list_match_participants[idx]

    """ fetch match_timeline """
    match_info = watcher.match.timeline_by_match(region=var_region,match_id=var_match_id)
    match_timeline = pd.json_normalize(data=match_info['info']['frames'], record_path='events')

    """ add match_id to match_timeline"""
    match_timeline['match_id'] = var_match_id

    """ convert match_timeline' timestamp to elapsed minutes"""
    match_timeline['timestamp_timedelta'] = pd.to_timedelta(match_timeline['timestamp'], unit='ms')
    match_timeline['elapsed_minutes'] = round(match_timeline['timestamp_timedelta']/ pd.Timedelta(minutes=1),3)
    
    """ subset for item_purchased """
    item_purchased = match_timeline.query("type == 'ITEM_PURCHASED'").dropna(axis=1)

    """ retrieve item_name """
    item_purchased['item_id'] = item_purchased['itemId'].astype(int)
    item_purchased['item_name'] = [dict_item_id_item_name[str(item)] for item in item_purchased['item_id']]

    """ retrieve summoner_puuid """
    item_purchased['participant_id'] = item_purchased['participantId'].astype(int)
    item_purchased['summoner_puuid'] = [dict_participants_puuids[str(id)] for id in item_purchased['participant_id']]

    """ subset for @my_puuid """
    item_purchased = item_purchased.query("summoner_puuid == @my_puuid")

    """ select cols - elapsed_minutes, item_name, summoner_puuid """
    item_purchased = item_purchased[
        [
            'match_id', 
            'elapsed_minutes',
            'item_name',
            'summoner_puuid'
        ]
    ]
    
    """ create dict with item_purchased per timestamp per participant (summoner) """
    dict_item_purchased = item_purchased.to_dict(orient='records')
    list_match_item_purchased.append(dict_item_purchased)

# record end time
end_timer = time.time()

# number of API calls
num_api_calls_per_cycle = 1
num_api_cycles = len(last_matches_puuid)
num_api_calls = num_api_cycles * num_api_calls_per_cycle

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 14.254740476608276 s
Number of API calls: 100
API calls per second: 7.02


In [5]:
""" obtain avg. elapsed_minutes per item_name (last matches)"""
from itertools import chain

df_match_item_purchased = pd.DataFrame(list(chain.from_iterable(list_match_item_purchased)))

avg_min_Dorans_Ring = df_match_item_purchased.query('item_name == "Doran\'s Ring"')['elapsed_minutes'].describe()
avg_min_Dorans_Ring

count    24.000000
mean      0.259708
std       0.146596
min       0.097000
25%       0.155500
50%       0.209000
75%       0.322000
max       0.614000
Name: elapsed_minutes, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 1. need to test first for API call before proceeding (hitting the API's rate limits as it is) 
# 2. need to add order_num to item_name to distinguish between similar item_names bought at different timestamps
# 3. .describe() to be applied per item_name_order_num 